# Evoke Tutorial

## 1. Introduction

+ [Tutorial](https://colab.research.google.com/drive/1zFRb20KoQi0Tdg5UR1_zReHCZicnQLZd#forceEdit=true&sandboxMode=true) **You are here!** | *Google Colab*
+ [Documentation](https://evoke.readthedocs.io/en/latest/) | *ReadTheDocs*
+ [Package](https://pypi.org/project/evoke-signals/) | *PyPI*
+ [Source code](https://github.com/signalling-games-org/evoke) | *GitHub*

**Evoke** is a Python library for evolutionary simulations of signalling games. It is particularly oriented towards reproducing results and figures from the literature, and offers a simple and intuitive API.

## 2. Set up

Install Evoke using pip.
This can take a little while because package dependencies will be installed too.

In [ ]:
%%capture

# Install evoke
!pip install evoke_signals

## 3. Creating signalling games

### Defining game parameters

You can use Evoke to create your own signalling games and evolutionary simulations.
Let's create a very simple 3x3 signalling game with two players.
We need four pieces of information to create the game:

+ `state_chances`: The number of states the sender can observe, and their probabilities. For this example there will be three equiprobable states.
+ `sender_payoff_matrix` and `receiver_payoff_matrix`: Matrices defining the payoffs of sender and receiver. For this example they will be the same: when the state matches the act, both agents get a payoff of 1, otherwise they get a payoff of zero.
+ `messages`: The number of messages available for the sender to send, which in this example is three.

In [ ]:
import numpy as np

state_chances = np.array([1/3, 1/3, 1/3])
sender_payoff_matrix = np.eye(3)
receiver_payoff_matrix = np.eye(3)
messages = 3

Let's have a look at one of those payoff matrices:

In [ ]:
sender_payoff_matrix

The rows correspond to states and the columns to acts.
So when the first act is performed in the first state, the agent gets a payoff of 1, and ditto for the second and third states and acts.
Any other combination of state and act yields a payoff of zero.

### Creating the game object

Sender-receiver games have a state that is determined by chance.
We therefore need to import the `Chance` class from the `games` module.

In [ ]:
from evoke.src.games import Chance

We can now create the game object,
This is an instance of the `Chance` class, with the game parameters fed into it.

In [ ]:
game = Chance(
    state_chances   = state_chances,
    sender_payoff_matrix = sender_payoff_matrix,
    receiver_payoff_matrix = receiver_payoff_matrix,
    messages = messages
)

### Creating the simulation object

A game is a static object.
It doesn't do anything on its own.
To make something happen, we need a set of **strategies** and a means by which those strategies **evolve** according to the payoffs they bring to the agents.

Fortunately, the game object already knows all the strategies that are in principle available to an agent:

In [ ]:
sender_strategies = game.sender_pure_strats()
receiver_strategies = game.receiver_pure_strats()

Let's imagine that some senders always send the same signal no matter the state, while others choose a different signal for each state.
These correspond to the first and sixth sender strategies (indexed by 0 and 5, because python indexing starts from 0):

In [ ]:
sender_strategies[0] # Pure strategy 0: always send signal 1.

In [ ]:
sender_strategies[5] # Pure strategy 5: for all i, send signal i in state i.

So, we will construct a population with just these two strategies, along with the corresponding strategies for the receiver:

In [ ]:
sender_population = np.array(
    [sender_strategies[0],sender_strategies[5]] # An array of strategies i.e. an array of matrices
)

receiver_population = np.array(
    [receiver_strategies[0],receiver_strategies[5]] # An array of strategies i.e. an array of matrices
)

...and we can evolve a population playing some mixture of these strategies using an appropriate simulation object from the `evolve` module:

In [ ]:
from evoke.src.evolve import TwoPops # One population of senders, one population of receivers

# Create the simulation object
evo = TwoPops(game, sender_population, receiver_population)

Let's see what happens when we evolve these populations.

In [ ]:
# Define equiprobable strategies
sender_strategy_vector = receiver_strategy_vector = np.array([1/2,1/2])

# Get a population vector in the format evo expects it
population_vector = np.concatenate((sender_strategy_vector, receiver_strategy_vector))

# For 100 iterations, get the new population vector
population_vectors_over_time = np.array([population_vector]) # this will store information about how the population changes
for _ in range(100):

    # Get the population vector at the next step
    population_vector = evo.discrete_replicator_delta_X(population_vector)

    # Store the population vector
    population_vectors_over_time = np.vstack((population_vectors_over_time,np.array([population_vector])))

Now `population_vectors_over_time` is a big list of how many of each type of sender and receiver there was at each step of the simulation.
Let's plot the change in the two sender types:

In [ ]:
from matplotlib import pyplot as plt

# Get the proportions of each sender type as it changes over time
sender_type_ignorant_time_series = population_vectors_over_time.T[0]
sender_type_responsive_time_series = population_vectors_over_time.T[1]

# Create the plot
plt.plot(range(len(population_vectors_over_time)),sender_type_ignorant_time_series,label="Ignorant sender")
plt.plot(range(len(population_vectors_over_time)),sender_type_responsive_time_series,label="Responsive sender")

# Add a legend
plt.legend()

# Add axis labels
plt.xlabel("Generations")
plt.ylabel("Proportion of sender type")

# Show the plot
plt.show()

The ignorant sender very quickly drops to zero, while the responsive sender very quickly dominates.

## 4. Recreating figures from the literature

### Creating figures; tweaking cosmetic features

Suppose you want to recreate Figure 1.1 from page 11 of *Signals* (Skyrms 2010).
The figure depicts the evolutionary dynamics of a 2x2 sender-receiver game.
The x-axis gives the proportion of receivers mapping the first signal to the second act and the second signal to the first act.
The y-axis gives the proportion of senders mapping the first state to the second signal and the second state to the first signal.

The points at which the population is achieving the greatest coordination are thus (0,0) and (1,1), so we would expect to see the arrows in the grid pointing towards those two corners.
Skyrms's figure shows exactly that.

For copyright reasons we can't show the original figure here.
Fortunately, recreating the figure is as easy as importing the relevant class and creating an instance of it:

In [ ]:
# Import the class
from evoke.examples.skyrms2010signals import Skyrms2010_1_1

# Create an instance of the class
fig1_1 = Skyrms2010_1_1()

If you check page 11 of _Signals_ you will see this plot closely matches Figure 1.1.

Let's say you don't like the boring black arrows and want them to be blue instead.
The figure you just created has various customisable options.
Change the `color` attribute, and Evoke will automatically rebuild the plot:

In [ ]:
fig1_1.color = "blue"

Now suppose you're appalled at the lack of axis labels.
You can add them like standard class attributes.
The figure would again be instantly recreated...

In [ ]:
fig1_1.show_immediately = False # Let's not create two new plots...
fig1_1.xlabel = "Proportion of receivers playing R2"
fig1_1.ylabel = "Proportion of senders playing R2"

...except that we suppressed the immediate output upon changing an attribute by first setting `fig1_1.show_immediately = False`.
Without this, the code would have created two new figures, one after the setting of `fig1_1.xlabel` and one after `fig1_1.ylabel`.

To show the figure manually, just call `show()`:

In [ ]:
fig1_1.show() #... let's just create one manually.

Some figures look better without axes at all.
Skyrms' second figure is an example.
It depicts the same 2x2 game, this time with a single population of agents that can play either sender or receiver in each round.
The evolutionary dynamics belong in a tetrahedron; the two stable points are at two vertices, with one or the other of the two strategies dominating:

In [ ]:
from evoke.examples.skyrms2010signals import * # import all figures from Skyrms

fig1_2 = Skyrms2010_1_2() # Create figure 1.2

If you want to see the axes, just switch `noaxis` to `False`:

In [ ]:
fig1_2.noaxis = False

I know what you're thinking, and the answer is yes, you can make the arrows blue.

In [ ]:
fig1_2.color = "blue"

To get a list of all the editable properties of a figure along with their current values, look at the `properties` property.
In addition to `noaxis` and `color`, we see that we can add axis labels to all three axes:

In [ ]:
fig1_2.properties

### Changing data in figures

One of the useful features of Evoke is that it allows you to re-run existing figures with different data.
In this way you can see how the results of a simulation would change if the parameters were tweaked.

Let's take figure 3.3 of Skyrms (2010:40) as an example.
This figure plots the increase in information carried by a signal over time, with agents using reinforcement learning to develop coordinated signalling and response strategies.

Once again we can create the basic figure just by creating an instance of the object:

In [ ]:
fig3_3 = Skyrms2010_3_3()

(Figures like this rely on randomisation, so the figure above might look a little different from how it does in the book. You can run the code block multiple times to get a sense of the possible variations.)

The basic figure shows what happens after 100 iterations.
Let's say we want to see what happens after 1000:

In [ ]:
fig3_3 = Skyrms2010_3_3(iterations=1000)

Even if 100 iterations was not enough to generate appreciable information transmission between agents, 1000 iterations very likely will be.

## 5. Adding to the stock of figures

You might want to add figures from the literature that aren't part of Evoke's example library yet.
If so, thanks!
We hope to expand the set of examples so that Evoke can become a place to test modelling assumptions and play around with data.

The following are simple steps to contribute to Evoke.
For more detailed instructions see the dedicated [CONTRIBUTING.md](https://github.com/signalling-games-org/evoke/blob/main/CONTRIBUTING.md) document.

### Fork the repository

First, fork the [Evoke](https://github.com/signalling-games-org/evoke) repository.

### Create a script

To add an example, create a new script whose name is the same format as the others in the `evoke/examples/` directory i.e. the first author's surname, the year, and the first word of the publication's title, all in lower case and without breaks or punctuation.
So for example Skyrms's book *Signals*, published in 2010, becomes `skyrms2010signals.py`.
The paper 'Communication and Common Interest' by Godfrey-Smith and Martínez becomes `godfreysmith2013communication.py`.

### Subclass from `figure.py`

Inside your script, create an object with the name formatted `SurnameYear_figure_number`.
For example, figure 4.1 of Skyrms (2010) is called `Skyrms2010_4_1`.

This object **must** subclass from one of the classes defined in `figure.py`.
If necessary, add a new class definition in `figure.py` first.

### Add custom code

Add your custom code in the object you just created.
Take a look at the existing examples to get a feel for how this works.

### Pull request

When you've tested and everything looks good, create a pull request from your fork.